In [1]:
import pandas as pd

In [2]:
file_path = r"/home/glenn/jupyter/precamplsia/PreclampsiaData.csv"
df = pd.read_csv(file_path)

In [3]:
df.head(2)

,Age,Pre_pregnancy_Weight,Parity,Annual_Income,Socioeconomic_Status,History_of_Hypertension,History_of_Diabetes,History_of_Preeclampsia,Proteinuria_Levels,Month,Systolic_BP,Diastolic_BP,Preclampsia_Outcome
0,19,74.9,1,0-284040,Low,No,No,No,1.0,1,151,88,1
1,19,73.9,1,0-284040,Low,Yes,No,Yes,3.0,2,125,76,0


In [4]:
df["History_of_Hypertension"] = df["History_of_Hypertension"].map({"No": 0, "Yes": 1})
df["History_of_Diabetes"] = df["History_of_Diabetes"].map({"No": 0, "Yes": 1})
df["History_of_Preeclampsia"] = df["History_of_Preeclampsia"].map({"No": 0, "Yes": 1})

#Socioeconomic Status 
df["Socioeconomic_Status"] = df["Socioeconomic_Status"].map({"Low": 0, "Medium": 1, "High": 2})

# Confirm the changes
print(df[["History_of_Hypertension", "History_of_Diabetes", "History_of_Preeclampsia", "Socioeconomic_Status"]].head(10))

   History_of_Hypertension  History_of_Diabetes  History_of_Preeclampsia  \
0                        0                    0                        0   
1                        1                    0                        1   
2                        0                    0                        0   
3                        0                    1                        0   
4                        1                    0                        1   
5                        0                    0                        0   
6                        1                    0                        1   
7                        0                    0                        0   
8                        0                    1                        0   
9                        0                    1                        0   

   Socioeconomic_Status  
0                     0  
1                     0  
2                     0  
3                     0  
4                     0  
5      

In [5]:
print(df.dtypes)

Age                          int64
Pre_pregnancy_Weight       float64
Parity                       int64
Annual_Income               object
Socioeconomic_Status         int64
History_of_Hypertension      int64
History_of_Diabetes          int64
History_of_Preeclampsia      int64
Proteinuria_Levels         float64
Month                        int64
Systolic_BP                  int64
Diastolic_BP                 int64
Preclampsia_Outcome          int64
dtype: object


In [6]:
# Annual Income into numerical categories
df["Annual_Income"] = df["Annual_Income"].replace({
    "0-284040": 1,
    "284041-1439988": 2,
    "Above 1439988": 3
}).astype(int)

#conversion
print(df["Annual_Income"].unique())
print(df.dtypes)

[1 2 3]
Age                          int64
Pre_pregnancy_Weight       float64
Parity                       int64
Annual_Income                int64
Socioeconomic_Status         int64
History_of_Hypertension      int64
History_of_Diabetes          int64
History_of_Preeclampsia      int64
Proteinuria_Levels         float64
Month                        int64
Systolic_BP                  int64
Diastolic_BP                 int64
Preclampsia_Outcome          int64
dtype: object


/tmp/ipykernel_735685/4034116019.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Annual_Income"] = df["Annual_Income"].replace({


In [7]:
X = df.drop(columns=["Preclampsia_Outcome"]) 
y = df["Preclampsia_Outcome"]

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

log_model = LogisticRegression()
log_model.fit(X_train, y_train)

y_pred_log = log_model.predict(X_test)

print("🔹 Logistic Regression Performance:")
print(classification_report(y_test, y_pred_log))

🔹 Logistic Regression Performance:
              precision    recall  f1-score   support

           0       0.90      0.91      0.90       424
           1       0.75      0.73      0.74       161

    accuracy                           0.86       585
   macro avg       0.83      0.82      0.82       585
weighted avg       0.86      0.86      0.86       585



/home/glenn/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

print("🔹 Random Forest Performance:")
print(classification_report(y_test, y_pred_rf))

🔹 Random Forest Performance:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       424
           1       0.97      0.97      0.97       161

    accuracy                           0.98       585
   macro avg       0.98      0.98      0.98       585
weighted avg       0.98      0.98      0.98       585



In [11]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)

print("🔹 XGBoost Performance:")
print(classification_report(y_test, y_pred_xgb))

/home/glenn/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:54:34] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


🔹 XGBoost Performance:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       424
           1       0.97      0.96      0.97       161

    accuracy                           0.98       585
   macro avg       0.98      0.98      0.98       585
weighted avg       0.98      0.98      0.98       585



In [12]:
from sklearn.svm import SVC

svm_model = SVC(kernel="linear")
svm_model.fit(X_train, y_train)

y_pred_svm = svm_model.predict(X_test)

print("🔹 SVM Performance:")
print(classification_report(y_test, y_pred_svm))

🔹 SVM Performance:
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       424
           1       0.80      0.88      0.83       161

    accuracy                           0.90       585
   macro avg       0.87      0.90      0.88       585
weighted avg       0.91      0.90      0.91       585



In [13]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

y_pred_dt = dt_model.predict(X_test)

print("🔹 Decision Tree Performance:")
print(classification_report(y_test, y_pred_dt))

🔹 Decision Tree Performance:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       424
           1       0.97      0.96      0.97       161

    accuracy                           0.98       585
   macro avg       0.98      0.98      0.98       585
weighted avg       0.98      0.98      0.98       585



In [14]:
# Import necessary libraries
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define base models
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)),
    ('svm', SVC(probability=True, kernel='rbf', random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42))
]

# Define meta-learner (final model)
meta_model = LogisticRegression()

# Create the stacking classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# Split the dataset into training and testing
X = df.drop(columns=["Preclampsia_Outcome"])  # Features
y = df["Preclampsia_Outcome"]  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the stacking model
stacking_model.fit(X_train, y_train)

# Predict on test data
y_pred = stacking_model.predict(X_test)

# Evaluate accuracy
stacking_accuracy = accuracy_score(y_test, y_pred)
print(f"Stacking Classifier Accuracy: {stacking_accuracy:.4f}")

/home/glenn/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:54:36] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/glenn/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:54:39] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/glenn/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:54:39] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/glenn/.local/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [12:54:39] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/glenn/.local/lib/python3.10/site-packages/xgboost/training.py:

Stacking Classifier Accuracy: 0.9829


In [15]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("classifier.pkl","wb")
pickle.dump(stacking_model, pickle_out)
pickle_out.close()

In [16]:
stacking_model.predict([[20,54.0,1,1,1,0,0,0,1.5,2,144,88]])

/home/glenn/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/glenn/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/home/glenn/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1])

In [17]:
stacking_model.predict([[20,71.0,1,1,1,0,0,0,0.3,2,105,67]])

/home/glenn/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/glenn/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/home/glenn/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([0])